In [2]:
import chromadb
import chromadb.utils.embedding_functions as embedding_functions
import numpy as np
import pandas as pd
import regex as re
import os

from langchain.prompts import ChatPromptTemplate

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings.openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

BASE_DIR = os.path.abspath(os.path.join(os.pardir))
DATA_DIR = os.path.abspath(os.path.join(BASE_DIR,"data"))

# Load DATA

In [6]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(os.path.join(DATA_DIR,"code_civil.pdf"))
pages = loader.load_and_split()

In [26]:
print(pages[40].page_content)

Titre préliminaire : De la publication, des effets et de l'application des lois en général
> Civ., 15 mai 2019, n° 18-12.602 (P)   [ ECLI:FR:CCASS:2019:C100434  ]
Dictionnaire du Droit privé
> Immeuble
4    
  Legif.   
  Plan   
  Jp.C.Cass.   
  Jp.Appel   
  Jp.Admin.   
  Juricaf
Le juge qui refusera de juger, sous prétexte du silence, de l'obscurité ou de l'insuffisance de la loi, pourra être
poursuivi comme coupable de déni de justice.
Récemment au Bulletin de la Cour de Cassation
> Civ., 25 octobre 2018, n° 17-16.828 (P)   [ ECLI:FR:CCASS:2018:C300955  ]
Dictionnaire du Droit privé
> Citation
5    
  Legif.   
  Plan   
  Jp.C.Cass.   
  Jp.Appel   
  Jp.Admin.   
  Juricaf
Il est défendu aux juges de prononcer par voie de disposition générale et réglementaire sur les causes qui leur
sont soumises.
Récemment au Bulletin de la Cour de Cassation
> Soc., 16 février 2022, n° 20-21.758, (B), FS   [ ECLI:FR:CCASS:2022:SO00234  ]
> Civ., 9 janvier 2019, n° 18-50.032 (P)   [ ECLI:FR:CCA

In [115]:
text = pages[500].page_content

In [116]:
print(text)

Livre III : Des différentes manières dont on acquiert la propriété - Titre III : Des sources d'obligations - Sous-titre II : La responsabilité extracontractuelle 
Le recours du fournisseur contre le producteur obéit aux mêmes règles que la demande émanant de la victime
directe du défaut. Toutefois, il doit agir dans l'année suivant la date de sa citation en justice.
1245-7  Ordonnance n°2016-131 du 10 février 2016 - art. 2   -  Conseil Constit. 2023-1045 QPC      
  Legif.   
  Plan   
  Jp.C.Cass.   
  Jp.Appel   
  Jp.Admin.   
  Juricaf
En cas de dommage causé par le défaut d'un produit incorporé dans un autre, le producteur de la partie
composante et celui qui a réalisé l'incorporation sont solidairement responsables.
1245-8  Ordonnance n°2016-131 du 10 février 2016 - art. 2   -  Conseil Constit. 2023-1045 QPC      
  Legif.   
  Plan   
  Jp.C.Cass.   
  Jp.Appel   
  Jp.Admin.   
  Juricaf
Le demandeur doit prouver le dommage, le défaut et le lien de causalité entre le défaut et 

In [109]:
re.findall("(?:\n)(\d+-?\d*)",text,re.DOTALL)

['4', '5', '6', '6-1', '6-2']

In [92]:
text

"Titre préliminaire : De la publication, des effets et de l'application des lois en général\n> Civ., 15 mai 2019, n° 18-12.602 (P)   [ ECLI:FR:CCASS:2019:C100434  ]\nDictionnaire du Droit privé\n> Immeuble\n4    \n  Legif.   \n  Plan   \n  Jp.C.Cass.   \n  Jp.Appel   \n  Jp.Admin.   \n  Juricaf\nLe juge qui refusera de juger, sous prétexte du silence, de l'obscurité ou de l'insuffisance de la loi, pourra être\npoursuivi comme coupable de déni de justice.\nRécemment au Bulletin de la Cour de Cassation\n> Civ., 25 octobre 2018, n° 17-16.828 (P)   [ ECLI:FR:CCASS:2018:C300955  ]\nDictionnaire du Droit privé\n> Citation\n5    \n  Legif.   \n  Plan   \n  Jp.C.Cass.   \n  Jp.Appel   \n  Jp.Admin.   \n  Juricaf\nIl est défendu aux juges de prononcer par voie de disposition générale et réglementaire sur les causes qui leur\nsont soumises.\nRécemment au Bulletin de la Cour de Cassation\n> Soc., 16 février 2022, n° 20-21.758, (B), FS   [ ECLI:FR:CCASS:2022:SO00234  ]\n> Civ., 9 janvier 2019, n° 

In [245]:
articles = []
for p in pages : 
    text = p.page_content
    articles += re.findall("(?:\n|^)\d+-?\d*.*?(?=\n\d+-?\d*|\np\. \d+)",text,re.DOTALL)

print(len(articles))

3318


In [246]:
pattern_to_remove = "\n  Legif.   \n  Plan   \n  Jp.C.Cass.   \n  Jp.Appel   \n  Jp.Admin.   \n  Juricaf"
articles = [re.sub(pattern_to_remove,"",a) for a in articles]
articles = [re.findall("(?:\n|^)\d+-?\d*.*?\n(.*)",a,re.DOTALL) for a in articles]
articles = [a[0] for a in articles if len(a)==1]

articles = [re.findall("[^\W\d]+",a.lower()) for a in articles]
articles = [[w for w in a if w not in sw]for a in articles]
articles = [" ".join(a) for a in articles]



Statistics

In [204]:
import nltk
from nltk.corpus import stopwords

In [205]:
nltk.download('stopwords')
sw = stopwords.words("french")


[nltk_data] Downloading package stopwords to
[nltk_data]     /lre/home/apellet/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [249]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = articles


In [250]:

vectorizer = CountVectorizer(stop_words=sw)
X = vectorizer.fit_transform(corpus)
vectorizer.get_feature_names_out()
#print(X.toarray())


array(['ab', 'abandon', 'abandonnant', ..., 'îlots', 'œuvre', 'œuvrer'],
      dtype=object)

In [251]:
M = pd.DataFrame(data=X.toarray(),columns = vectorizer.get_feature_names_out())

In [252]:
M.sum(axis=0).sort_values(ascending=False)[:50]

peut            1462
fr              1324
droit           1031
être             998
ccass            973
ecli             973
si               774
article          768
chambre          763
cas              671
civile           660
privé            603
dictionnaire     552
acte             514
personne         505
cour             501
cassation        496
époux            491
bulletin         489
récemment        487
lorsque          487
juge             458
biens            454
celui            424
cette            389
autre            384
fait             377
sans             367
enfant           367
ci               353
première         350
titre            343
tout             342
civil            340
état             338
doit             336
sous             335
contrat          333
droits           330
entre            328
peuvent          326
lieu             305
dont             296
demande          291
conditions       291
mariage          282
dispositions     277
articles     

In [253]:
from top2vec import Top2Vec

In [223]:
corpus[45]

"Il est tenu compte pour la détermination, à toute époque, du territoire français, des modifications résultant des\nactes de l'autorité publique française pris en application de la Constitution et des lois, ainsi que des traités\ninternationaux survenus antérieurement."

In [254]:
model = Top2Vec(documents=corpus,embedding_model="doc2vec",speed="deep-learn")

2024-11-19 11:10:20,732 - top2vec - INFO - Pre-processing documents for training
2024-11-19 11:10:20,933 - top2vec - INFO - Creating joint document/word embedding
2024-11-19 11:11:36,508 - top2vec - INFO - Creating lower dimension embedding of documents
2024-11-19 11:11:42,334 - top2vec - INFO - Finding dense areas of documents
2024-11-19 11:11:42,416 - top2vec - INFO - Finding topics


In [255]:
model.get_num_topics()

31

In [258]:
topic_words, word_scores, topic_nums = model.get_topics(31)


In [229]:
topic_words

array([['financiere', 'co', 'ecli', ..., 'relative', 'cause', 'foi'],
       ['protegee', 'tuteur', 'gestion', ..., 'prejudice', 'succession',
        'reduction'],
       ['dette', 'creancier', 'creance', ..., 'dernier', 'immeubles',
        'non'],
       ...,
       ['prescription', 'delai', 'compter', ..., 'economique',
        'immeuble', 'mai'],
       ['ouvrage', 'dommages', 'responsabilite', ..., 'associes',
        'exercer', 'ccass'],
       ['compensatoire', 'prestation', 'capital', ..., 'avril',
        'execution', 'deuxieme']], dtype='<U14')

In [259]:
df = pd.DataFrame(topic_words).T